## 0. Load imports 

In [98]:
## imports
import pandas as pd
import numpy as np


## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 0. Load data

In [77]:
## load data on 2020 crimes in DC
dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")
dc_crim_2020.head()
dc_crim_2020.shape
dc_crim_2020.info()


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.044885,38.911506,20024652,2020/02/10 05:06:34+00,MIDNIGHT,GUN,ROBBERY,1600 - 1639 BLOCK OF 20TH STREET NW,396107.33,138182.60,...,005502 1,5502.0,Precinct 14,38.911498,-77.044883,DUPONT CIRCLE,2020/02/10 04:09:47+00,2020/02/10 05:06:40+00,97431273,NaN
1,-77.046632,38.912245,20016623,2020/01/28 04:01:40+00,MIDNIGHT,OTHERS,THEFT F/AUTO,1613 - 1699 BLOCK OF 21ST STREET NW,395955.84,138264.80,...,005502 1,5502.0,Precinct 14,38.912238,-77.046630,NaN,2020/01/28 01:00:07+00,2020/01/28 03:01:00+00,97431275,NaN
2,-77.044497,38.915343,20027213,2020/02/14 07:38:05+00,MIDNIGHT,OTHERS,THEFT F/AUTO,1907 - 1999 BLOCK OF FLORIDA AVENUE NW,396141.17,138608.55,...,004202 2,4202.0,Precinct 14,38.915335,-77.044495,NaN,2020/02/14 05:40:50+00,2020/02/14 06:30:11+00,97431276,NaN
3,-77.046639,38.907581,20083061,2020/06/04 23:18:23+00,EVENING,OTHERS,THEFT/OTHER,1301 - 1319 BLOCK OF 21ST STREET NW,395954.99,137747.05,...,005502 2,5502.0,Precinct 14,38.907573,-77.046637,NaN,2020/06/04 00:30:00+00,2020/06/04 12:00:03+00,97431278,NaN
4,-77.045303,38.907659,20117215,2020/08/16 23:27:28+00,EVENING,OTHERS,BURGLARY,1300 - 1348 BLOCK OF NEW HAMPSHIRE AVENUE NW,396070.88,137755.60,...,005502 3,5502.0,Precinct 14,38.907651,-77.045301,NaN,2020/08/16 03:30:21+00,2020/08/16 06:00:27+00,97431285,NaN


(27915, 25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27915 entries, 0 to 27914
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     27915 non-null  float64
 1   Y                     27915 non-null  float64
 2   CCN                   27915 non-null  int64  
 3   REPORT_DAT            27915 non-null  object 
 4   SHIFT                 27915 non-null  object 
 5   METHOD                27915 non-null  object 
 6   OFFENSE               27915 non-null  object 
 7   BLOCK                 27915 non-null  object 
 8   XBLOCK                27915 non-null  float64
 9   YBLOCK                27915 non-null  float64
 10  WARD                  27915 non-null  int64  
 11  ANC                   27915 non-null  object 
 12  DISTRICT              27886 non-null  float64
 13  PSA                   27880 non-null  float64
 14  NEIGHBORHOOD_CLUSTER  27915 non-null  object 
 15  BLOCK_GROUP        

## 1. Aggregation

### Example of grouping by one variable and doing one aggregation 

**Task**: find and print the number of unique offense types (`OFFENSE`) by ward (`WARD`)

In [78]:
## way one: use built in unique function
unique_off_byward = dc_crim_2020.groupby('WARD').agg({'OFFENSE': 'nunique'}).reset_index()
unique_off_byward

## way two: use lambda function
unique_off_byward_v2 = dc_crim_2020.groupby('WARD').agg({'OFFENSE': lambda x: len(x.unique())}).reset_index()
unique_off_byward_v2

,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


### Example of grouping by one variable and providing two summaries of the same variable

**Task**: previous showed number of offenses by ward but want to find out content of offenses in each ward; create an aggregation that summarizes both the number of unique offenses by ward and what those offenses are -- for instance, by pasting the unique offenses in that ward separated by the ";" (e.g., Theft; Burglary;...)

*Hint*: you can use the join command to paste together a list separated by some delimiter. 
    - The syntax, if we are using the comma delimiter, is: ", ".join(nameoflist)

In [79]:
## one way to solve -- write out the whole function inside the agg and use default varnames
unique_off_valuecount_byward = dc_crim_2020.groupby('WARD').agg({'OFFENSE': ['nunique',
                                            lambda x: "; ".join(sorted(x.unique()))]}).reset_index()

unique_off_valuecount_byward


## a second way to solve --- write the function outside of the agg and then feed that function
## to the agg (still uses lambda)
def find_join_unique(x):
    
    sorted_un = sorted(x.unique())
    joined_un = "; ".join(sorted_un)
    return(joined_un)

unique_off_valuecount_byward_v2 = dc_crim_2020.groupby('WARD').agg({'OFFENSE': ['nunique',
                                            lambda x: find_join_unique(x)]}).reset_index()


## a third way to solve --- give more informative column names by using a diff structure
## inside agg
unique_off_valuecount_byward_bettername = dc_crim_2020.groupby("WARD").agg(
                                                            count_offense =
                                                            ('OFFENSE', lambda x: 
                                                             len(x.unique())), 
                                                            name_offense =
                                                            ('OFFENSE', lambda x: 
                                                             find_join_unique(x))).reset_index()

unique_off_valuecount_byward_bettername

WARD OFFENSE                                                   
       nunique                                         <lambda_0>
0    1       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
1    2       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
2    3       8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
3    4       8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
4    5       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
5    6       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
6    7       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
7    8       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...

,WARD,count_offense,name_offense
0,1,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
1,2,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
2,3,8,ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
3,4,8,ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
4,5,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
5,6,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
6,7,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
7,8,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...


### Example of grouping by two variables 

**Task**: group by ward (`WARD`) and police shift (`SHIFT`) and find the offense that is most common in that ward and shift

In [80]:
## way one
top_w_shift = dc_crim_2020.groupby(['WARD', 'SHIFT']).agg({'OFFENSE': lambda x: 
                                                               x.value_counts(sort = True, 
                                                            ascending = False).index[0]}).reset_index()
top_w_shift

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


In [81]:
## way two: function defined outside the pandas dataframe (preview for next section)

def most_common(one_col: pd.Series):
    
    ## sort values
    sorted_series = one_col.value_counts(sort = True, ascending = False)
    
    ## get top 
    top = sorted_series.index[0]
    
    ## return
    return(top)

In [82]:
top_w_shift_alternate = dc_crim_2020.groupby(['WARD', 'SHIFT']).agg({'OFFENSE': 
                                                lambda x: most_common(x)}).reset_index()

In [83]:
top_w_shift_alternate

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


## Summarizing over all rows or all columns (without grouping)

We can also use the `apply` function to summarize rows or columns efficiently

**Task**: find the mean lat and longitude in one line of code

In [84]:
## pandas built-in
dc_crim_2020[["LATITUDE", "LONGITUDE"]].apply('mean', axis = 0)

## numpy
dc_crim_2020[["LATITUDE", "LONGITUDE"]].apply(np.mean, axis = 0)


LATITUDE     38.907616
LONGITUDE   -77.007033
dtype: float64

LATITUDE     38.907616
LONGITUDE   -77.007033
dtype: float64

In [85]:
## what happens if we change the axis parameter from
## axis = 0 to axis = 1?

dc_crim_2020[["LATITUDE", "LONGITUDE"]].apply(np.mean, axis = 1) # just taking mean across 
# rows which isn't informative 

0       -19.066692
1       -19.067196
2       -19.064580
3       -19.069532
4       -19.068825
           ...    
27910   -19.058034
27911   -19.055430
27912   -19.031056
27913   -19.035166
27914   -19.031056
Length: 27915, dtype: float64

## 2. Creating new columns/transforming their type 

### Simple np.where

`np.where` is in the numpy package (aliased as `np`) and operates similar to `ifelse` in R

**Task**: create an indicator `is_theft` for any offense that contains the word "THEFT"
    
**Task**: create an indicator `is_theft_notmotor` for any offense that contains the word "THEFT" but does not contain the word "MOTOR"

In [86]:

## approach 1: np.where 
dc_crim_2020['is_theft'] = np.where(dc_crim_2020.OFFENSE.str.contains("THEFT"), True, False)

## approach 2: list iteration 
## returning True if theft is in
## offense, false otherwise
## note that offense is an arbitrary placeholder
## and could be replaced with x, o, etc.
dc_crim_2020['is_theft_alternate'] = [True if "THEFT" in offense
                                      else False
                                      for offense in dc_crim_2020.OFFENSE]

pd.crosstab(dc_crim_2020.is_theft, dc_crim_2020.OFFENSE)
pd.crosstab(dc_crim_2020.is_theft, dc_crim_2020.is_theft_alternate)


## then, create variable for whether it is a non-motor theft
dc_crim_2020['is_theft_notmotor'] = np.where(dc_crim_2020.OFFENSE.str.contains("THEFT") & 
                                            ~dc_crim_2020.OFFENSE.str.contains("MOTOR"), 
                                             True, False)


pd.crosstab(dc_crim_2020.is_theft_notmotor, dc_crim_2020.OFFENSE)

OFFENSE,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER
is_theft,,,,,,,,,
False,13,1630,1441,195,0,1998,177,0,0
True,0,0,0,0,3260,0,0,8275,10926


is_theft_alternate,False,True
is_theft,,
False,5454,0
True,0,22461


OFFENSE,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER
is_theft_notmotor,,,,,,,,,
False,13,1630,1441,195,3260,1998,177,0,0
True,0,0,0,0,0,0,0,8275,10926


### np.select

**Task**: create a new variable, `offense_summary`, where you:
        
- Recode theft offenses that use a gun or knife as the method (`METHOD`) as: violent theft
- Recode non-theft offenses that use a gun or knife as the method as: violent other
- Recode all other as non-violent 

In [87]:
## for convenience, copy to shorter name
df = dc_crim_2020.copy()

## step 1: create conditions
conditions_offense = [df['is_theft'] & df['METHOD'].isin(['GUN', 'KNIFE']),
                     ~df['is_theft'] & df['METHOD'].isin(['GUN', 'KNIFE'])]

## step 2: create what each condition resolves/codes to
code_to = ['violent theft', 'violent other']

## step 3: feed np.select the list of 
## conditions and what they code to
df['offense_summary'] = np.select(conditions_offense,
                                 code_to, 
                                 default = 'non-violent')

## double check outputs
df.offense_summary.value_counts()

## check
df.loc[(df.is_theft) &
  (df.METHOD.isin(['GUN', 'KNIFE'])),
['OFFENSE', 'offense_summary', 'METHOD']].head()


non-violent      25241
violent other     2639
violent theft       35
Name: offense_summary, dtype: int64

,OFFENSE,offense_summary,METHOD
696,MOTOR VEHICLE THEFT,violent theft,KNIFE
1312,THEFT/OTHER,violent theft,GUN
2278,THEFT/OTHER,violent theft,GUN
2932,THEFT/OTHER,violent theft,KNIFE
3770,THEFT/OTHER,violent theft,KNIFE


### map.recode

**Task**: recode shifts that are MIDNIGHT or EVENING as "nighttime"; code other shift to daytime

In [88]:
shifts_dict = {'MIDNIGHT': 'nighttime',
              'EVENING': 'nighttime'}

dc_crim_2020['summary_shifts'] = dc_crim_2020.SHIFT.map(shifts_dict).fillna("daytime")

pd.crosstab(dc_crim_2020.summary_shifts,
           dc_crim_2020.SHIFT)

SHIFT,DAY,EVENING,MIDNIGHT
summary_shifts,,,
daytime,10092,0,0
nighttime,0,12315,5508


### Using built-in pandas methods

Another way of creating variables, used either alone or in combination with np.where and np.select is to use pandas built in `str` methods

Basic structure is: `df['namestringcol'].str.someoperation`

**Task**: using a `str` method, create a new variable--`OFFENSE_NOSP`--that replaces spaces in the `OFFENSE` column with underscores

In [89]:
dc_crim_2020['OFFENSE_NOSP'] = dc_crim_2020.OFFENSE.str.replace(" ", "_")

## print example of random sample of ones that should
## be changed; detect this via str.contains
dc_crim_2020.loc[dc_crim_2020.OFFENSE.str.contains(" "),
            ["OFFENSE", "OFFENSE_NOSP"]].sample(n = 10)

,OFFENSE,OFFENSE_NOSP
24217,THEFT F/AUTO,THEFT_F/AUTO
23086,THEFT F/AUTO,THEFT_F/AUTO
3030,THEFT F/AUTO,THEFT_F/AUTO
20395,MOTOR VEHICLE THEFT,MOTOR_VEHICLE_THEFT
13812,ASSAULT W/DANGEROUS WEAPON,ASSAULT_W/DANGEROUS_WEAPON
24866,THEFT F/AUTO,THEFT_F/AUTO
6129,THEFT F/AUTO,THEFT_F/AUTO
8277,ASSAULT W/DANGEROUS WEAPON,ASSAULT_W/DANGEROUS_WEAPON
20074,MOTOR VEHICLE THEFT,MOTOR_VEHICLE_THEFT
13553,THEFT F/AUTO,THEFT_F/AUTO


### Transforming column types

**Task**: check the type of the `START_DATE` column

In [90]:
dc_crim_2020.START_DATE.dtype

dtype('O')

**Task**: recast the `START_DATE` column to datetime, calling this `START_DATE_CLEAN`; coerce errors rather than cleaning the string; check the type of that new col

In [91]:
dc_crim_2020["START_DATE_CLEAN"] = pd.to_datetime(dc_crim_2020.START_DATE, 
                                                 errors = "coerce")

dc_crim_2020.START_DATE_CLEAN.dtype

datetime64[ns, UTC]

**Task:** print the min and max of `START_DATE_CLEAN`. What happens if you try to do this with `START_DATE`?

In [92]:
dc_crim_2020.START_DATE_CLEAN.min()
dc_crim_2020.START_DATE_CLEAN.max()


Timestamp('1940-02-17 00:00:16+0000', tz='UTC')

Timestamp('2021-01-01 02:30:21+0000', tz='UTC')

## 3. Row and column filtering 

### Row subsetting

**Task**: filter to crime reports about theft using the `is_theft` definition and that are in ward 3

In [93]:
theft_w3 = dc_crim_2020[(dc_crim_2020.is_theft) &
                       (dc_crim_2020.WARD == 3)].copy()

## one way to check: assert
assert all(theft_w3.is_theft == True)
assert all(theft_w3.WARD == 3)

## another way to check: value counts
theft_w3.is_theft.value_counts()
theft_w3.WARD.value_counts()

True    1521
Name: is_theft, dtype: int64

3    1521
Name: WARD, dtype: int64

### Column subsetting


**Task**: select two columns--`START_DATE` and `END_DATE`--and print the head of the dataframe

In [94]:
dc_crim_2020[['START_DATE', 'END_DATE']].head()

,START_DATE,END_DATE
0,2020/02/10 04:09:47+00,2020/02/10 05:06:40+00
1,2020/01/28 01:00:07+00,2020/01/28 03:01:00+00
2,2020/02/14 05:40:50+00,2020/02/14 06:30:11+00
3,2020/06/04 00:30:00+00,2020/06/04 12:00:03+00
4,2020/08/16 03:30:21+00,2020/08/16 06:00:27+00


**Task**: use list comprehension to automate this a bit and select all columns with the word "DATE" in the name; print the head of the dataframe

In [95]:
dc_crim_2020[[col for col in dc_crim_2020.columns if 
             "DATE" in col]].head()

,START_DATE,END_DATE,START_DATE_CLEAN
0,2020/02/10 04:09:47+00,2020/02/10 05:06:40+00,2020-02-10 04:09:47+00:00
1,2020/01/28 01:00:07+00,2020/01/28 03:01:00+00,2020-01-28 01:00:07+00:00
2,2020/02/14 05:40:50+00,2020/02/14 06:30:11+00,2020-02-14 05:40:50+00:00
3,2020/06/04 00:30:00+00,2020/06/04 12:00:03+00,2020-06-04 00:30:00+00:00
4,2020/08/16 03:30:21+00,2020/08/16 06:00:27+00,2020-08-16 03:30:21+00:00


**Task**: filter reports of crime in `START_DATE_CLEAN` before May 2020 and that are located in ward 1; select the ward and `START_DATE` column and print a random sample of rows

In [96]:
dc_crim_2020.loc[(dc_crim_2020.START_DATE_CLEAN < "2020/05/01") &
                (dc_crim_2020.WARD == 1),
                ["START_DATE", "WARD"]].sample(n = 10)


,START_DATE,WARD
17322,2020/03/26 12:20:19+00,1
15556,2019/12/10 05:00:00+00,1
1354,2020/02/11 18:06:57+00,1
25511,2020/04/11 05:13:46+00,1
18012,2020/04/21 19:10:55+00,1
18473,2020/01/30 04:54:44+00,1
1783,2020/02/20 17:38:39+00,1
17808,2020/02/13 23:44:28+00,1
3251,2020/03/03 15:00:58+00,1
19490,2020/04/19 18:00:45+00,1
